## Extraction the Harry Potter related entities

- Characters: Harry Potter, Hermione Granger, Albus Dumbledore, etc.

- Locations: Hogwarts, Diagon Alley, Hogsmeade, etc.

- Organizations: Ministry of Magic, Order of the Phoenix, Death Eaters, etc.

- Magical Objects: Elder Wand, Invisibility Cloak, Philosopher's Stone, etc.

- Spells and Potions: Expelliarmus, Polyjuice Potion, etc.

- Events: Triwizard Tournament, Battle of Hogwarts, etc.

- Creatures: Hippogriff, Basilisk, Dementor, etc.

In [54]:
import requests
import re
import pandas as pd

from bs4 import BeautifulSoup

In [55]:
# Spells
spells = requests.get("https://hp-api.onrender.com/api/spells").json()
spell_names = [spell["name"] for spell in spells]

In [56]:
# Characters
characters = requests.get("https://hp-api.onrender.com/api/characters").json()
character_names = [character["name"] for character in characters]

In [57]:
url = "https://harrypotter.fandom.com/wiki/List_of_creatures"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Send GET request
response = requests.get(url, headers=headers)
response.raise_for_status()  # Check for HTTP errors

soup = BeautifulSoup(response.text, 'html.parser')
content = soup.find('div', {'class': 'mw-parser-output'})

creatures_data = []

# Find all section headings (h2 with span.mw-headline)
for section in content.find_all('h2'):
    # Extract section title
    headline = section.find('span', class_='mw-headline')
    if not headline:
        continue
    
    section_title = headline.text.strip()
    # print(section_title)
    
    # Skip non-creature sections
    if section_title == 'Notes and references':
        continue
    
    # Find the corresponding creatures list
    creatures_ul = section.find_next('ul', )
    # print(creatures_ul)
    if not creatures_ul:
        continue
    
    # Extract creature names
    for li in creatures_ul.find_all('li'):

        li_ul = li.find_all('ul')
        if li_ul:
            for li_li in li_ul:
                if isinstance(li_li, list):
                    for sub_li in li_li.find_all('li'):
                        creature = sub_li.text.strip()
                        # Remove reference numbers like [1], [5], etc.
                        creature = re.sub(r'\[\d+\]', '', creature)
                        creature = re.sub(r'\s*\(.*?\)', '', creature)
                        creatures_data.append({
                            'Section': section_title,
                            'Creature': creature
                        })
                else:
                    creature = li_li.text.strip()
                    # Remove reference numbers like [1], [5], etc.
                    creature = re.sub(r'\[\d+\]', '', creature)
                    creature = re.sub(r'\s*\(.*?\)', '', creature)
                    creatures_data.append({
                        'Section': section_title,
                        'Creature': creature
                    })
            next
        creature = li.text.strip()
        # Remove reference numbers like [1], [5], etc.
        creature = re.sub(r'\[\d+\]', '', creature)
        creature = re.sub(r'\s*\(.*?\)', '', creature)
        creatures_data.append({
            'Section': section_title,
            'Creature': creature
        })

# Print the data    
for creature in creatures_data:
    print(creature['Creature'])

Acromantula
Ashwinder
Augurey
Basilisk
Billywig
Blast-Ended Skrewt
Bowtruckle
Bundimun
Catoblepas
Centaur
Chimaera
Chizpurfle
Chupacabra
Clabbert
Coruscant cuttlefish
Crup
Demiguise
Diricawl
Double-ended Newt
Doxy
Antipodean Opaleye
Catalonian Fireball
Chinese Fireball
Common Welsh Green
Hebridean Black
Hungarian Horntail
Norwegian Ridgeback
Peruvian Vipertooth
Portuguese Long-Snout
Romanian Longhorn
Swedish Short-Snout
Ukrainian Ironbelly
Water dragon
Dragon
Antipodean Opaleye
Catalonian Fireball
Chinese Fireball
Common Welsh Green
Hebridean Black
Hungarian Horntail
Norwegian Ridgeback
Peruvian Vipertooth
Portuguese Long-Snout
Romanian Longhorn
Swedish Short-Snout
Ukrainian Ironbelly
Water dragon
Antipodean Opaleye
Catalonian Fireball
Chinese Fireball
Common Welsh Green
Hebridean Black
Hungarian Horntail
Norwegian Ridgeback
Peruvian Vipertooth
Portuguese Long-Snout
Romanian Longhorn
Swedish Short-Snout
Ukrainian Ironbelly
Water dragon
Dragon Shark
Drake
Dugbog
Erkling
Erumpent
Fairy
G

In [ ]:
# drop duplicates by "Creature" column
creatures_df = pd.DataFrame(creatures_data)
creatures_df.drop_duplicates(subset='Creature', inplace=True)
creatures_df.reset_index(drop=True, inplace=True)
creatures_df_cleaned = creatures_df[~creatures_df['Creature'].str.contains('\n', na=False)]
creatures_df_cleaned.reset_index(drop=True, inplace=True)
print(creatures_df_cleaned)

                   Section        Creature
0                   Beasts     Acromantula
1                   Beasts       Ashwinder
2                   Beasts         Augurey
3                   Beasts        Basilisk
4                   Beasts        Billywig
..                     ...             ...
284  Magic related animals   Tropical bird
285  Magic related animals  Tarantula hawk
286  Magic related animals       Wild Boar
287  Magic related animals        The Grim
288  Magic related animals  The White Wolf

[289 rows x 2 columns]
